In [46]:
import kagglehub
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
import pandas as pd
from networkx.algorithms.community import louvain_communities
from collections import Counter
import re
from collections import defaultdict


# Download latest version
path = kagglehub.dataset_download("harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows")
print("Path to dataset files:", path)

df = pd.read_csv(f"{path}/imdb_top_1000.csv")
# print(df[:2])
print(df.info())

Path to dataset files: /Users/karenstentoft/.cache/kagglehub/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows/versions/1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Poster_Link    1000 non-null   object 
 1   Series_Title   1000 non-null   object 
 2   Released_Year  1000 non-null   object 
 3   Certificate    899 non-null    object 
 4   Runtime        1000 non-null   object 
 5   Genre          1000 non-null   object 
 6   IMDB_Rating    1000 non-null   float64
 7   Overview       1000 non-null   object 
 8   Meta_score     843 non-null    float64
 9   Director       1000 non-null   object 
 10  Star1          1000 non-null   object 
 11  Star2          1000 non-null   object 
 12  Star3          1000 non-null   object 
 13  Star4          1000 non-null   object 
 14  No_of_Votes    1000 non-null   int64  
 1

In [47]:
df = df.drop(columns=["Poster_Link", "Meta_score", "Certificate"])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series_Title   1000 non-null   object 
 1   Released_Year  1000 non-null   object 
 2   Runtime        1000 non-null   object 
 3   Genre          1000 non-null   object 
 4   IMDB_Rating    1000 non-null   float64
 5   Overview       1000 non-null   object 
 6   Director       1000 non-null   object 
 7   Star1          1000 non-null   object 
 8   Star2          1000 non-null   object 
 9   Star3          1000 non-null   object 
 10  Star4          1000 non-null   object 
 11  No_of_Votes    1000 non-null   int64  
 12  Gross          831 non-null    object 
dtypes: float64(1), int64(1), object(11)
memory usage: 101.7+ KB
None


In [48]:
# df = df.dropna().reset_index(drop=True)
# print(df['Gross'])

# Add imdb id

In [60]:
import requests

#API_KEY = "2aaf18d5"  
# API_KEY = "d8f1a797"  
#API_KEY = "e61b1332"
API_KEY = "a8c3078"
url = f"http://www.omdbapi.com/?apikey={API_KEY}&t=Inception"



# def get_movie_info(title):
#     url = "http://www.omdbapi.com/"
#     params = {
#         "t": title,
#         "apikey": API_KEY
#     }
#     r = requests.get(url, params=params)
#     print(title, r.status_code, r.url)
#     return r.json()

def get_movie_info(title):
    url = "http://www.omdbapi.com/"
    params = {"t": title, "apikey": API_KEY}

    r = requests.get(url, params=params)
    print(title, r.status_code, r.url)

    # Debug output
    print("RAW RESPONSE:", repr(r.text[:200]))

    try:
        return r.json()
    except Exception as e:
        print("JSON decode failed:", e)
        return {"Response": "False", "Error": "Invalid JSON from API"}


In [61]:
df["imdbId"] = None

In [62]:
for idx, row in df.iterrows():
    title = row["Series_Title"]
    info = get_movie_info(title)
    if info.get("Response") == "True":
        df.at[idx, "imdbId"] = info.get("imdbID")

print(df.info())

The Shawshank Redemption 200 http://www.omdbapi.com/?t=The+Shawshank+Redemption&apikey=a8c3078
RAW RESPONSE: '{"Title":"The Shawshank Redemption","Year":"1994","Rated":"R","Released":"14 Oct 1994","Runtime":"142 min","Genre":"Drama","Director":"Frank Darabont","Writer":"Stephen King, Frank Darabont","Actors":'
The Godfather 200 http://www.omdbapi.com/?t=The+Godfather&apikey=a8c3078
RAW RESPONSE: '{"Title":"The Godfather","Year":"1972","Rated":"R","Released":"24 Mar 1972","Runtime":"175 min","Genre":"Crime, Drama","Director":"Francis Ford Coppola","Writer":"Mario Puzo, Francis Ford Coppola","Ac'
The Dark Knight 200 http://www.omdbapi.com/?t=The+Dark+Knight&apikey=a8c3078
RAW RESPONSE: '{"Title":"The Dark Knight","Year":"2008","Rated":"PG-13","Released":"18 Jul 2008","Runtime":"152 min","Genre":"Action, Crime, Drama","Director":"Christopher Nolan","Writer":"Jonathan Nolan, Christophe'
The Godfather: Part II 200 http://www.omdbapi.com/?t=The+Godfather%3A+Part+II&apikey=a8c3078
RAW RESPONS

In [63]:
df_raw = df.copy()

In [64]:
df = df_raw.dropna(subset = ["imdbId"])

In [65]:
df["imdbId"] = (
    df["imdbId"].str.replace("tt", "")
    .str.lstrip("0")
    .astype(int)
)

/var/folders/_w/g2kh2v8d5yxb4h0fkd7pblpm0000gn/T/ipykernel_57494/2553034127.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["imdbId"] = (


# Adding GroupLens data

In [66]:
df_links = pd.read_csv("../../ml-latest/links.csv")

df_links = df_links.drop(columns=["tmdbId"])

print(df_links)

       movieId    imdbId
0            1    114709
1            2    113497
2            3    113228
3            4    114885
4            5    113041
...        ...       ...
86532   288967  14418234
86533   288971  11162178
86534   288975     70199
86535   288977  23050520
86536   288983  11644948

[86537 rows x 2 columns]


In [67]:
df_combined = df.merge(df_links, on="imdbId")

print(df_combined.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 789 entries, 0 to 788
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series_Title   789 non-null    object 
 1   Released_Year  789 non-null    object 
 2   Runtime        789 non-null    object 
 3   Genre          789 non-null    object 
 4   IMDB_Rating    789 non-null    float64
 5   Overview       789 non-null    object 
 6   Director       789 non-null    object 
 7   Star1          789 non-null    object 
 8   Star2          789 non-null    object 
 9   Star3          789 non-null    object 
 10  Star4          789 non-null    object 
 11  No_of_Votes    789 non-null    int64  
 12  Gross          679 non-null    object 
 13  imdbId         789 non-null    int64  
 14  movieId        789 non-null    int64  
dtypes: float64(1), int64(3), object(11)
memory usage: 92.6+ KB
None


In [68]:
df_tags = pd.read_csv("../../ml-latest/tags.csv")

df_tags = df_tags.drop(columns= ["timestamp"])

df_tags_grouped = df_tags.groupby("movieId").agg(lambda x: list(x))
df_tags_grouped = df_tags_grouped.rename(columns={"userId": "userId_tags"})

In [69]:
df_combined2 = df_combined.merge(df_tags_grouped, on="movieId")

In [70]:
df_ratings = pd.read_csv("../../ml-latest/ratings.csv")
df_ratings = df_ratings.drop(columns= ["timestamp"])


df_ratings_grouped = df_ratings.groupby("movieId").agg(lambda x: list(x))
df_ratings_grouped = df_ratings_grouped.rename(columns={"userId": "userId_ratings"})


In [71]:
df_final = df_combined2.merge(df_ratings_grouped, on="movieId")

print(df_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Series_Title    782 non-null    object 
 1   Released_Year   782 non-null    object 
 2   Runtime         782 non-null    object 
 3   Genre           782 non-null    object 
 4   IMDB_Rating     782 non-null    float64
 5   Overview        782 non-null    object 
 6   Director        782 non-null    object 
 7   Star1           782 non-null    object 
 8   Star2           782 non-null    object 
 9   Star3           782 non-null    object 
 10  Star4           782 non-null    object 
 11  No_of_Votes     782 non-null    int64  
 12  Gross           676 non-null    object 
 13  imdbId          782 non-null    int64  
 14  movieId         782 non-null    int64  
 15  userId_tags     782 non-null    object 
 16  tag             782 non-null    object 
 17  userId_ratings  782 non-null    obj

In [72]:
df_final = df_final.rename(columns={
    "Series_Title": "Title",
    "IMDB_Rating": "imdb_rating",
    "Overview": "Description",
    "No_of_Votes": "imdb_no_of_votes",
    "imdbId": "imdb_id",
    "movieId": "gl_id",
    "userId_tags": "gl_user_id_tags",
    "tag": "gl_tag",
    "userId_ratings": "gl_user_id_ratings",
    "rating": "gl_rating"
})

In [73]:
df_final['Released_Year'] = (
    df_final['Released_Year']
    .astype(str)               # sikrer str
    .str.extract(r'(\d{4})')   # trækker første 4 cifre ud
    [0]                        # extract returnerer DataFrame → tag første kolonne
    .astype(float)             # konverter til float (så NaN tillades)
)


In [74]:
print(df_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Title               782 non-null    object 
 1   Released_Year       781 non-null    float64
 2   Runtime             782 non-null    object 
 3   Genre               782 non-null    object 
 4   imdb_rating         782 non-null    float64
 5   Description         782 non-null    object 
 6   Director            782 non-null    object 
 7   Star1               782 non-null    object 
 8   Star2               782 non-null    object 
 9   Star3               782 non-null    object 
 10  Star4               782 non-null    object 
 11  imdb_no_of_votes    782 non-null    int64  
 12  Gross               676 non-null    object 
 13  imdb_id             782 non-null    int64  
 14  gl_id               782 non-null    int64  
 15  gl_user_id_tags     782 non-null    object 
 16  gl_tag  

In [75]:
df_final["gl_user_id_tags"] = df_final["gl_user_id_tags"].apply(lambda lst: list(set(lst)))

In [76]:
print(df_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Title               782 non-null    object 
 1   Released_Year       781 non-null    float64
 2   Runtime             782 non-null    object 
 3   Genre               782 non-null    object 
 4   imdb_rating         782 non-null    float64
 5   Description         782 non-null    object 
 6   Director            782 non-null    object 
 7   Star1               782 non-null    object 
 8   Star2               782 non-null    object 
 9   Star3               782 non-null    object 
 10  Star4               782 non-null    object 
 11  imdb_no_of_votes    782 non-null    int64  
 12  Gross               676 non-null    object 
 13  imdb_id             782 non-null    int64  
 14  gl_id               782 non-null    int64  
 15  gl_user_id_tags     782 non-null    object 
 16  gl_tag  

In [77]:
df_final

,Title,Released_Year,Runtime,Genre,imdb_rating,Description,Director,Star1,Star2,Star3,Star4,imdb_no_of_votes,Gross,imdb_id,gl_id,gl_user_id_tags,gl_tag,gl_user_id_ratings,gl_rating
0,The Shawshank Redemption,1994.0,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",111161,318,"[73742, 229398, 319521, 270371, 327715, 180262...","[in, classic, drama, great acting, hope, inspi...","[2, 3, 4, 5, 6, 9, 11, 14, 15, 22, 26, 27, 30,...","[5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 4.0, 3.5, 3.5, ..."
1,The Godfather,1972.0,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",68646,858,"[229382, 231434, 272396, 274445, 88080, 116752...","[Mafia, too long, boring, atmospheric, Mafia, ...","[3, 4, 11, 12, 13, 15, 26, 30, 31, 33, 35, 40,...","[5.0, 3.0, 4.0, 3.0, 4.0, 5.0, 5.0, 5.0, 3.0, ..."
2,The Dark Knight,2008.0,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",468569,58559,"[116739, 100356, 249860, 92166, 83976, 180232,...","[Atmospheric, Batman, comic book, dark, Heath ...","[6, 10, 11, 14, 15, 17, 22, 30, 37, 40, 44, 49...","[5.0, 5.0, 4.5, 5.0, 3.0, 5.0, 4.0, 5.0, 5.0, ..."
3,The Godfather: Part II,1974.0,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",71562,1221,"[158208, 275458, 164360, 22026, 261643, 14, 17...","[Al Pacino, mafia, classic, complex characters...","[15, 26, 30, 35, 53, 57, 78, 86, 99, 104, 105,...","[5.0, 5.0, 4.5, 4.5, 4.5, 5.0, 5.0, 4.5, 5.0, ..."
4,12 Angry Men,1957.0,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000",50083,1203,"[231434, 77839, 231441, 20505, 112668, 151600,...","[group psychology, classic, courtroom, courtro...","[30, 31, 33, 44, 51, 76, 78, 96, 105, 112, 123...","[5.0, 2.0, 5.0, 5.0, 5.0, 4.5, 5.0, 0.5, 5.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,Breakfast at Tiffany's,1961.0,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN,54698,902,"[222223, 315416, 205338, 46623, 185887, 307234...","[Audrey Hepburn, Netflix Finland, 1960s, apart...","[48, 99, 137, 148, 149, 175, 181, 189, 203, 22...","[5.0, 5.0, 4.0, 2.0, 3.5, 5.0, 2.5, 3.5, 4.0, ..."
778,Giant,1956.0,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN,49261,948,"[8833, 83717, 270499, 7716, 215490, 241095, 23...","[National Film Registry, on dvr, epic, Classic...","[148, 481, 490, 548, 927, 1678, 1767, 1783, 18...","[3.0, 3.0, 4.0, 5.0, 4.0, 3.0, 3.0, 3.0, 4.5, ..."
779,From Here to Eternity,1953.0,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000",45793,1944,"[238092, 222223, 126357, 210197, 226209, 7716,...","[National Film Registry, Nana viu, Deborah Ker...","[94, 149, 442, 444, 481, 487, 525, 548, 562, 6...","[4.5, 2.5, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, 4.0, ..."
780,Lifeboat,1944.0,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN,37017,2202,"[88449, 172802, 190434, 108132, 215490, 229729...","[reviewed, Hitchcock, holes40s, tivo, history,...","[105, 149, 301, 663, 927, 1057, 1156, 1306, 13...","[5.0, 3.0, 4.0, 3.5, 4.0, 4.5, 4.0, 4.0, 3.0, ..."


# Save to csv

In [ ]:
df_final.to_csv("Merged_data.csv", index=False)

In [ ]:
# Creating the graph
G = nx.Graph()

for i in range(len(df_final)):
    G.add_node(df_final['Title'][i],
    release_year = int(df_final['Released_Year'][i]) if not pd.isna(df_final['Released_Year'][i]) else None,
    runtime = int((df_final['Runtime'][i]).replace(' min', '')),
    genre = (df_final['Genre'][i]).replace(" ", "").split(','),
    imdb_rating = float(df_final['imdb_rating'][i]),
    description = df_final['Description'][i],
    director = df_final['Director'][i],
    stars = [df_final['Star1'][i], df_final['Star2'][i], df_final['Star3'][i], df_final['Star4'][i]],
    imdb_no_of_votes = int(df_final['imdb_no_of_votes'][i]),
    gross = int((df_final['Gross'][i]).replace(',', '')) if not pd.isna(df_final['Gross'][i]) else None,
    imdb_id = df_final["imdb_id"][i],
    gl_id = df_final["gl_id"][i],
    gl_user_id_tags = df_final["gl_user_id_tags"][i],
    gl_tag = df_final["gl_tag"][i],
    gl_user_id_ratings = df_final["gl_user_id_ratings"][i],
    gl_rating = df_final["gl_rating"][i]
    )

print(G)



Graph with 438 nodes and 0 edges
